# Simple Object Detection in Tensorflow
This lab will walk through how to use object detection models available in Tensorflow Hub.


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

### Download the model from Tensorflow Hub
Tensorflow Hub is a repository of trained machine learning models which can be reuses in projects.

In [2]:

# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# can choose ssd mobilenet version 2 instead
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

### Load the model
 load the model specified by the module_handle.

In [3]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2025-08-13 16:06:23.397350: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-08-13 16:06:23.397418: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-08-13 16:06:23.397436: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-08-13 16:06:23.397449: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-13 16:06:23.397459: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-08-13 16:06:23.956295: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_regi

### Choose the default signature
Some models in the Tensorflow hub can be used for different tasks. So each model's documentation should show what signature to use when running the model.

If one wants to see if a model has more than one signature then one can do something like print(hub.load(module_handle).signatures.keys()). In my case, the models I use only have the default signature so I don't have to worry about other types.

In [4]:
# take a look at the available signatures for this particular model
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction () -> Dict[['detection_scores', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)], ['detection_class_names', TensorSpec(shape=(None, 1), dtype=tf.string, name=None)], ['detection_class_entities', TensorSpec(shape=(None, 1), dtype=tf.string, name=None)], ['detection_boxes', TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)], ['detection_class_labels', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None)]] at 0x32D5B7290>}))

### choose the 'default' signature for object detector.

For object detection models, its 'default' signature will accept a batch of image tensors and output a dictionary describing the objects detected, which is what I want here.

In [5]:
detector = model.signatures['default']

### download_and_resize_image
This function downloads an image specified by a given "url", pre-processes it, and then saves it to disk.

In [6]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Fetches an image online, resizes it and saves it locally.

    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image

    Returns:
        (string) -- path to the saved image
    '''


    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")

    # opens the given URL
    response = urlopen(url)

    # reads the image fetched from the URL
    image_data = response.read()

    # puts the image data in memory buffer
    image_data = BytesIO(image_data)

    # opens the image
    pil_image = Image.open(image_data)

    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.Resampling.LANCZOS)

    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")

    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)

    print("Image downloaded to %s." % filename)

    return filename

### Download and preprocess an image
Now, using download_and_resize_image I get a sample image online and save it locally.

In [7]:
# can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /var/folders/9k/fjwgdp8s2bd7ywp5b9yssgw00000gn/T/tmpqhdr320w.jpg.


### run_detector
This function takes in the object detection model detector and the path to a sample image, then use this model to detect objects and display its predicted class categories and detection boxes.

run_detector uses load_image to convert the image into a tensor.

In [8]:
def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.

    Args:
        path (string) -- path to a locally saved JPEG image

    Returns:
        (tensor) -- an image tensor
    '''

    # read the file
    img = tf.io.read_file(path)

    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)

    return img


def run_detector(detector, path):
    '''
    Runs inference on a local file using an object detection model.

    Args:
        detector (model) -- an object detection model loaded from TF Hub
        path (string) -- path to an image saved locally
    '''

    # load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


### Run inference on the image
I run detector by calling the run_detector function. This prints the number of objects found followed by three lists:

- The detection scores of each object found (i.e. how confident the model is),
- The classes of each object found,
- The bounding boxes of each object

In [9]:
run_detector(detector, downloaded_image_path)

W0000 00:00:1755094471.283161  483424 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -2484 } dim { size: -2485 } dim { size: -2486 } dim { size: 1088 } } } inputs { dtype: DT_FLOAT shape { dim { size: -105 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -105 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 17 } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 12 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { dim { size: -105 } dim { size: 17 } dim { size: 17 } dim { size: 1088 } } 

Found 100 objects.
[0.65321696 0.61050385 0.6015246  0.592552   0.5917802  0.5815478
 0.5505323  0.4957591  0.47425103 0.4732213  0.44066203 0.40511316
 0.3980361  0.39406416 0.37148595 0.36155915 0.36150882 0.34689036
 0.33362255 0.31253242 0.2887808  0.2575848  0.25749162 0.2519611
 0.24781859 0.23412514 0.20431808 0.20324862 0.17988189 0.17965089
 0.17374797 0.16431478 0.16031054 0.15895325 0.1562039  0.15468822
 0.14754567 0.13622591 0.12740251 0.1255574  0.12102627 0.11813003
 0.11387341 0.11229258 0.11129044 0.09718847 0.09137164 0.08975992
 0.08880346 0.08633563 0.08337443 0.08095174 0.07988743 0.07741397
 0.07731989 0.07631022 0.07507869 0.07386067 0.07233311 0.07204021
 0.07110172 0.06935565 0.06825575 0.06427943 0.06248567 0.06226338
 0.06211052 0.0594006  0.05798876 0.05784502 0.0572553  0.05346744
 0.05304386 0.05242869 0.04892577 0.04813002 0.04578408 0.04424202
 0.04337543 0.04277803 0.04262635 0.04163408 0.04081724 0.03976008
 0.03945999 0.03944283 0.03863989 0.03768988 